In [ ]:
# lecture8: 记载数据集，分batch训练
## 这玩意不收敛啊
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
 
# prepare dataset
 
 
class DiabetesDataset(Dataset):
    def __init__(self, filepath):
        xy = np.loadtxt(filepath, delimiter=',', dtype=np.float32)
        self.len = xy.shape[0] # shape(多少行，多少列)
        self.x_data = torch.from_numpy(xy[:, :-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
 
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
 
    def __len__(self):
        return self.len
 
 
dataset = DiabetesDataset('diabetes.csv')
print(len(dataset))
train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=0) #num_workers 多线程,我这里可能是因为这个代码的版本太低，用不了，后面查查1.8.0的dataloader怎么用
 
 
# design model using class
 
 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = torch.nn.Linear(8, 6)
        self.linear2 = torch.nn.Linear(6, 4)
        self.linear3 = torch.nn.Linear(4, 1)
        self.sigmoid = torch.nn.Sigmoid()
 
    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        x = self.sigmoid(self.linear3(x))
        return x
 
 
model = Model()
 
# construct loss and optimizer
criterion = torch.nn.BCELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
epoch_list = []
loss_list = []
# training cycle forward, backward, update
# 分batch训练，本质上一个batch采用的就是向量化的读入，然后len是759，所以我们这里每一个epoch有24轮，这里轮与轮之间采用的是随机梯度下降法
if __name__ == '__main__':
    for epoch in range(100):
        for i, data in enumerate(train_loader, 0): # train_loader 是先shuffle后mini_batch
            inputs, labels = data
            y_pred = model(inputs)
            loss = criterion(y_pred, labels)
            #print("epoch", epoch, "这个epoch里的第几轮", i, loss.item())
 
            optimizer.zero_grad()
            loss.backward()
 
            optimizer.step()
        epoch_list.append(epoch)
        loss_list.append(loss.item())
plt.plot(epoch_list, loss_list)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [9]:
759/32


23.71875

In [19]:
# lecture9 多分类问题，mnist demo
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
 
# prepare dataset
 
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) # 归一化,均值和方差，对于mnist数据集的均值和方差就这么取
 
train_dataset = datasets.MNIST(root='../dataset/mnist/', train=True, download=True, transform=transform)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='../dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# 注意训练集和测试集的预处理是一样的，正常应当是在训练集上算出方差以及均值，然后到test里面应用 
# design model using class
 
print(train_loader)
 
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)
 
    def forward(self, x):
        x = x.view(-1, 784)  # -1其实就是自动获取mini_batch  28*28
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)  # 最后一层不做激活，不进行非线性变换,因为我们用的交叉墒损失函数
 
 
model = Net()
 
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 
# training cycle forward, backward, update
 
 
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        optimizer.zero_grad()
        # 获得模型预测结果(64, 10)
        outputs = model(inputs)
        # 交叉熵代价函数outputs(64,10),target（64）
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            print(outputs)
            _, predicted = torch.max(outputs.data, dim=1) # dim = 1 列是第0个维度，行是第1个维度
            ## 上面一句代码很有解释的必要：
            ### 我们注意网络设置最后输出10个feature，下标表示就是0到9，样本数应当是一个batchsize
            ### 所以，我们输出概率值最大的那个feature对应的下标就是他所属的类别（下标0到9对应类别0到9）
            ### 根据下面的demo code，就是predicted
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
            break
    print('accuracy on test set: %d %% ' % (100*correct/total))     #由于是百分比所以乘100
 
 
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 2.221
[1,   600] loss: 0.921
[1,   900] loss: 0.436
tensor([[ 3.0435e+00, -6.0041e+00,  1.4306e+00,  2.0450e+00, -4.4995e+00,
          1.1751e-01, -3.9453e+00,  8.3674e+00, -1.4784e+00,  2.8725e+00],
        [ 2.6392e+00, -4.3600e+00,  9.5705e+00,  4.4671e+00, -6.4068e+00,
          3.1838e+00,  6.0244e+00, -3.7834e+00,  1.9783e+00, -5.9935e+00],
        [-4.8030e+00,  5.0517e+00,  1.5010e+00,  8.8099e-01, -1.5064e+00,
         -8.6250e-01, -1.3852e+00,  4.8142e-01,  1.5402e+00, -3.3241e-01],
        [ 1.4241e+01, -1.5737e+01,  5.5798e+00,  1.7092e+00, -9.5529e+00,
          6.2581e+00,  5.5532e+00,  1.9526e+00,  1.4458e+00, -1.2876e+00],
        [-2.1227e+00, -4.4828e+00,  6.9753e-01, -2.2091e+00,  6.3460e+00,
         -1.5243e+00,  2.7836e-01,  2.0756e-03,  7.4786e-01,  3.9041e+00],
        [-6.2769e+00,  6.4047e+00,  2.0358e+00,  1.2288e+00, -1.7480e+00,
         -1.6317e+00, -2.7884e+00,  1.3546e+00,  2.1196e+00,  1.0621e-01],
        [-2.9480e+00, -4.0524e+00, -2

In [8]:
test = torch.rand(3,2)
test2,test3 = torch.max(test,dim=1)
print(test)
print(test2)
print(test3)

tensor([[0.8119, 0.4852],
        [0.3829, 0.9152],
        [0.0246, 0.3244]])
tensor([0.8119, 0.9152, 0.3244])
tensor([0, 1, 1])


In [18]:
employee_names = ["Paul", "John", "Abbie", "Charlotte", "Ron"]

for index, name in enumerate(employee_names):
	print(index, name)

0 Paul
1 John
2 Abbie
3 Charlotte
4 Ron


In [19]:
employee_names = ["Paul", "John", "Abbie", "Charlotte", "Ron"]


for index, name in enumerate(employee_names, 1):
	print(index, name)

1 Paul
2 John
3 Abbie
4 Charlotte
5 Ron
